In [2]:
#!/usr/bin/env python3
import rospy
from interbotix_xs_msgs.msg import JointSingleCommand
from control_msgs.msg import JointControllerState


In [ ]:
#!/usr/bin/env python3
import rospy
import actionlib
from sensor_msgs.msg import JointState
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from control_msgs.msg import FollowJointTrajectoryAction, FollowJointTrajectoryGoal
from actionlib_msgs.msg import GoalStatus

class SafeGripper:
    def __init__(self, effort_thresh=1000.0):
        rospy.init_node("safe_gripper")

        # 1) Effort monitor
        self.effort_thresh = effort_thresh
        self.curr_effort = 0.0
        rospy.Subscriber(
            "/locobot/dynamixel/joint_states",
            JointState,
            self._joint_states_cb,
            queue_size=1
        )

        # 2) Action client for the gripper controller
        self.client = actionlib.SimpleActionClient(
            "/locobot/gripper_controller/follow_joint_trajectory",
            FollowJointTrajectoryAction
        )
        rospy.loginfo("Waiting for gripper action server…")
        self.client.wait_for_server()
        rospy.loginfo("Gripper action server ready")

    def _joint_states_cb(self, msg: JointState):
        # map names→effort
        efforts = dict(zip(msg.name, msg.effort))
        # take the max of the two fingers
        self.curr_effort = max(
            abs(efforts.get("left_finger", 0.0)),
            abs(efforts.get("right_finger", 0.0))
        )

    def close(self, target_width: float, duration: float = 1.0):
        # Build a 1-point trajectory to close the gripper
        goal = FollowJointTrajectoryGoal()
        goal.trajectory.joint_names = ["left_finger", "right_finger"]
        p = JointTrajectoryPoint()
        # Note: for your setup `pos` may need to be [ w/2, -w/2 ] 
        # to open/close symmetrically
        p.positions = [target_width/2, -target_width/2]
        p.time_from_start = rospy.Duration(duration)
        goal.trajectory.points = [p]

        # Send the goal, but DON'T wait_for_result yet
        self.client.send_goal(goal)
        rate = rospy.Rate(100)  # 100Hz monitor loop

        # 3) Monitor until done or effort spike
        while not rospy.is_shutdown():
            state = self.client.get_state()
            if state not in (GoalStatus.ACTIVE, GoalStatus.PENDING):
                # trajectory finished (success or failure)
                break

            if self.curr_effort > self.effort_thresh:
                rospy.logwarn(f"Effort {self.curr_effort:.1f} > {self.effort_thresh}, cancelling!")
                self.client.cancel_goal()
                break

            rate.sleep()

        final_state = self.client.get_state()
        if final_state == GoalStatus.SUCCEEDED:
            rospy.loginfo("Gripper closed successfully")
        else:
            rospy.loginfo(f"Gripper action ended with state {final_state}")

if __name__ == "__main__":
    sg = SafeGripper(effort_thresh=300.0)
    rospy.sleep(0.5)
    # Example: close to 0.1 rad in 1s, but will abort if effort >1000
    sg.close(target_width=0.1, duration=1.0)


In [ ]:
open_cmd = JointSingleCommand(name="gripper", cmd=0.05)
gd.pub.publish(open_cmd)
rospy.sleep(0.5)
# Then close until grasp
gd.close_until_grasp()